# Using BFGP++ in the Unified Planning framework

## Installation
(Show how to install the interface with pip commands)

## Execution modes

(Short description of the 3 modes accepted in the system)
(Run the following examples from Python code instead of parsing PDDL)
(Describe the Gripper example before the modes)

First let's import the required libraries:

In [27]:
from unified_planning.shortcuts import *  # type: ignore
from unified_planning.plans import PlanKind  # type: ignore
from unified_planning.engines.results import PlanGenerationResultStatus

The execution of the framework requires a `domain_file` name, a list of `problem_files` names and the set of arguments (`args`) which depend on the executed mode. The following code serve as a base for calling _BFGP++_:

In [28]:
def base_bfgp_call(domain_file: str, problem_files: List[str], args: dict):
    with up.environment.get_environment().factory.FewshotPlanner(name='bfgp') as bfgp:
        bfgp.set_arguments(**args)
        problems = bfgp.generate_problems(domain_file, problem_files)
        # Compute the generalized plan for these input problems
        result = bfgp.solve(problems, output_stream=sys.stdout)
        # Check whether all generated plans are satisficing
        return all(r == PlanGenerationResultStatus.SOLVED_SATISFICING for r in result)

This function assumes the domain and problems are existing files, however, they can also be implemented in Python code as shown in many other usage examples in the Unified Planning library. Once the list of `problems` is implemented (instead of parsed), _BFGP++_ can be directly called with the `bfgp.solve(problems)` command. 



### Synthesis

This mode refers to the generation of a planning program given a domain and a set of planning instances in that domain. For instance, the _Gripper_ domain is defined by 3 sorts of objects (_room_, _ball_, and _gripper_); 4 predicates that refer to whether the robby is (_robby-at_), whether each ball is in a specific room (_at_), if a gripper is free (_free_) or if it is carrying a ball (_carry_); 2 sorts of constants which refer to _left_ and _right_ grippers, and rooms _A_ and _B_; and 3 action schemas to describe ...

In [29]:
    """BFGP++ synthesizing a program that can solve 3 different Gripper instances"""
    domain = '../tests/domains/gripper/domain.pddl'
    problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(1, 4)]
    kwargs = dict({'mode': 'synthesis',
                   'theory': 'cpp',
                   'program_lines': 10,
                   'program': 'gripper',
                   'translated_problem_dir': 'tmp_gripper/'})

    print(f"Does a solution exist for Gripper? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m synthesis -t cpp -f tmp_gripper/ -s 0 -l 10 -o tmp_gripper//gripper -pgp True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Evaluation function ed added.
[INFO] Evaluation function ilc added.
[INFO] Engine created. [0.000s]
[INFO] Searching...
[ENGINE]
Node id=0
Expanded=1
Evaluated=1
Open queue size=0
[NODE]: f1=1 f2=0
0. empty
1. empty
2. empty
3. empty
4. empty
5. empty
6. empty
7. empty
8. empty
9. end

[ENGINE]
Node id=1
Expanded=2
Evaluated=87
Open queue size=85
[NODE]: f1=1 f2=-1
0. for(ptr_ball_0++,8)
1. empty
2. empty
3. empty
4. empty
5. empty
6. empty
7. empty
8. endfor(ptr_ball_0++,0)
9. end

[ENGINE]
Node id=87
Expanded=3
Evaluated=145
Open queue size=142
[NODE]: f1=1 f2=-2
0. for(ptr_ball_0++,8)
1. for(ptr_gripper_0++,7)
2. empty
3. empty
4. empty

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m validation-prog -t cpp -f tmp_gripper/ -s 0 -p tmp_gripper//gripper.prog  -o tmp_gripper/  -plans True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Program loaded: [0.000s]
[INFO] Read program:
0. for(ptr_ball_0++,8)
1. for(ptr_room_0++,7)
2. for(ptr_room_1++,6)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
6. endfor(ptr_room_1++,2)
7. endfor(ptr_room_0++,1)
8. endfor(ptr_ball_0++,0)
9. end

[INFO] Number of instances: 3
[INFO] GOAL ACHIEVED!
[INFO] Total time: [0.001s]
Does a solution exist for Gripper? True



### Validation

In the `synthesis` mode example above, the Gripper solution is first found, then generates one plan per instance and validates those plans with the UP library. Now let's assume that we have the program, e.g. the Gripper solution, and we want to validate it over new planning instances, e.g. starting with 10 and 13 balls in room A, so we run the following command to validate the program:

In [30]:
"""BFGP++ validating a program over 2 different and larger Gripper instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(4, 6)]
kwargs = dict({'mode': 'validation-prog',
               'theory': 'cpp',
               'program': 'gripper',
               'translated_problem_dir': 'tmp_gripper/'})
print(f"Is the program valid over new Gripper instances? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m validation-prog -t cpp -f tmp_gripper/ -s 0 -p tmp_gripper//gripper.prog  -o tmp_gripper/  -plans True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Program loaded: [0.000s]
[INFO] Read program:
0. for(ptr_ball_0++,8)
1. for(ptr_room_0++,7)
2. for(ptr_room_1++,6)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
6. endfor(ptr_room_1++,2)
7. endfor(ptr_room_0++,1)
8. endfor(ptr_ball_0++,0)
9. end

[INFO] Number of instances: 2
[INFO] GOAL ACHIEVED!
[INFO] Total time: [0.001s]
Is the program valid over new Gripper instances? True


 

### Repair

The two modes above can be interpreted as if the user has no knowledge at all about the solution (empty program, hence run `synthesis`) or full knowledge about it (full program is known, hence run `validation`). Thus, `repair` mode is an intermediate step between `synthesis` and `validation`, where some parts of the program may be known (also they are maybe wrong) and the system must repair them to get to a valid solution if one exists.  The following examples try to repair 3 different input programs where the first is all correct, in the second all loops are missing, and in the third all planning actions are missing:

In [31]:
"""BFGP++ repairing (if needed) 3 Gripper programs for the same input instances"""
domain = '../tests/domains/gripper/domain.pddl'
problems = [f'../tests/domains/gripper/p{i:02}.pddl' for i in range(1, 4)]

# Example 1: the input program is correct
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_ok',
               'translated_problem_dir': 'tmp_gripper_ok/'})

print(f"Example 1 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

# Example 2: the input program has some missing loops
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_missing_loops',
               'translated_problem_dir': 'tmp_gripper_missing_loops/'})

print(f"Example 2 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

# Example 3: the input program has some missing planning action
kwargs = dict({'mode': 'repair',
               'theory': 'cpp',
               'program_lines': 10,
               'program': '../tests/domains/gripper/program_repair/gripper_cpp_missing_planning_actions',
               'translated_problem_dir': 'tmp_gripper_missing_planning_actions/'})

print(f"Example 3 has been repaired? {base_bfgp_call(domain_file=domain, problem_files=problems, args=kwargs)}")

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m repair -t cpp -f tmp_gripper_ok/ -s 0 -l 10  -p ../tests/domains/gripper/program_repair/gripper_cpp_ok.prog  -o tmp_gripper_ok/  -pgp True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Evaluation function ed added.
[INFO] Evaluation function ilc added.
[INFO] Engine created. [0.000s]
[INFO] Searching...
[INFO] SOLUTION FOUND!!!
0. for(ptr_ball_0++,6)
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. move(ptr_room_1,ptr_room_0)
6. endfor(ptr_ball_0++,0)
7. end

[INFO] Number of instances: 3
[INFO] Expanded: 0
[INFO] Evaluated: 1
[INFO] Total plan costs: 48
[INFO] Total number of instructions: 75
[INFO] Search time: [0.000s]
[INFO] Total time: [0.000s]
[INFO] Program file

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m validation-prog -t cpp -f tmp_gripper_ok/ -s 0 -p tmp_gripper_ok//gripper_cpp_ok.prog  -o tmp_gripper_ok/  -plans True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Program loaded: [0.000s]
[INFO] Read program:
0. for(ptr_ball_0++,6)
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. move(ptr_room_1,ptr_room_0)
6. endfor(ptr_ball_0++,0)
7. end

[INFO] Number of instances: 3
[INFO] GOAL ACHIEVED!
[INFO] Total time: [0.001s]
Example 1 has been repaired? True


/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m repair -t cpp -f tmp_gripper_missing_loops/ -s 0 -l 10  -p ../tests/domains/gripper/program_repair/gripper_cpp_missing_loops.prog  -o tmp_gripper_missing_loops/  -pgp True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Evaluation function ed added.
[INFO] Evaluation function ilc added.
[INFO] Engine created. [0.000s]
[INFO] Searching...
[ENGINE]
Node id=0
Expanded=1
Evaluated=6
Open queue size=5
[NODE]: f1=1 f2=0
0. empty
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. move(ptr_room_1,ptr_room_0)
6. empty
7. end

[INFO] Solution candidate!
[NODE]: f1=0 f2=-1
0. for(ptr_ball_0++,6)
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_ro

/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m validation-prog -t cpp -f tmp_gripper_missing_loops/ -s 0 -p tmp_gripper_missing_loops//gripper_cpp_missing_loops.prog  -o tmp_gripper_missing_loops/  -plans True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Program loaded: [0.000s]
[INFO] Read program:
0. for(ptr_ball_0++,6)
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. move(ptr_room_1,ptr_room_0)
6. endfor(ptr_ball_0++,0)
7. end

[INFO] Number of instances: 3
[INFO] GOAL ACHIEVED!
[INFO] Total time: [0.001s]
Example 2 has been repaired? True


/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m repair -t cpp -f tmp_gripper_missing_planning_actions/ -s 0 -l 10  -p ../tests/domains/gripper/program_repair/gripper_cpp_missing_planning_actions.prog  -o tmp_gripper_missing_planning_actions/  -pgp True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Evaluation function ed added.
[INFO] Evaluation function ilc added.
[INFO] Engine created. [0.000s]
[INFO] Searching...
[ENGINE]
Node id=0
Expanded=1
Evaluated=2
Open queue size=1
[NODE]: f1=1 f2=-1
0. for(ptr_ball_0++,6)
1. empty
2. empty
3. empty
4. empty
5. empty
6. endfor(ptr_ball_0++,0)
7. end

[ENGINE]
Node id=2
Expanded=2
Evaluated=48
Open queue size=46
[NODE]: f1=1 f2=-2
0. for(ptr_ball_0++,6)
1. for(ptr_gripper_0++,5)
2. empty
3. empty
4. empty
5. endfor(ptr_gripper_0++,1)
6. endfor(ptr_ball_0++,0)
7. end


/home/js/Desktop/software/AIPlan4EU/up-bfgp/up_bfgp/bfgp_pp/main.bin -m validation-prog -t cpp -f tmp_gripper_missing_planning_actions/ -s 0 -p tmp_gripper_missing_planning_actions//gripper_cpp_missing_planning_actions.prog  -o tmp_gripper_missing_planning_actions/  -plans True
[INFO] Parsed domain. [0.000s]
[INFO] Building theory: cpp
[INFO] Generalized Domain created. [0.000s]
[INFO] Infinite detection: deactivated
[INFO] Generalized Planning Problem created. [0.000s]
[INFO] Program loaded: [0.000s]
[INFO] Read program:
0. for(ptr_ball_0++,6)
1. pick(ptr_ball_0,ptr_room_0,ptr_gripper_0)
2. inc(ptr_room_1)
3. move(ptr_room_0,ptr_room_1)
4. drop(ptr_ball_0,ptr_room_1,ptr_gripper_0)
5. move(ptr_room_1,ptr_room_0)
6. endfor(ptr_ball_0++,0)
7. end

[INFO] Number of instances: 3
[INFO] GOAL ACHIEVED!
[INFO] Total time: [0.001s]
Example 3 has been repaired? True




## Input Arguments
(List the input arguments accepted by the interface)

## Usage Examples
(Short description of the 3 theories or target languages accepted in the system)



  

### STRIPS benchmarks
...

### Program Synthesis benchmarks
...

### Learning Action Models benchmarks
...

